In [1]:
!pip install pandas==1.5.3
!pip install sentence-transformers accelerate -U
!pip install transformers[torch] -U

import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2
Accelerate version: 0.27.2
Transformers version: 4.38.1


In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
import pandas as pd

epa_data = drive.CreateFile({'id': '1nOdK7YrAzx-2ZpKZCd620rbSG9qLLxZp'})
epa_data.GetContentFile('EPA_EmissionsData.csv')
factor_df = pd.read_csv('EPA_EmissionsData.csv')
print(factor_df.head())

activity_data = drive.CreateFile({'id': '1smUkjfTvvHmx2QiCfDImKOOtdCSb4dTk'})
activity_data.GetContentFile('business_activities_training_data.csv')
activity_df = pd.read_csv('business_activities_training_data.csv')
print(activity_df.head())

   2017 NAICS Code                  2017 NAICS Title       GHG  \
0           111110                   Soybean Farming  All GHGs   
1           111120  Oilseed (except Soybean) Farming  All GHGs   
2           111130          Dry Pea and Bean Farming  All GHGs   
3           111140                     Wheat Farming  All GHGs   
4           111150                      Corn Farming  All GHGs   

                                Unit  \
0  kg CO2e/2021 USD, purchaser price   
1  kg CO2e/2021 USD, purchaser price   
2  kg CO2e/2021 USD, purchaser price   
3  kg CO2e/2021 USD, purchaser price   
4  kg CO2e/2021 USD, purchaser price   

   Supply Chain Emission Factors without Margins  \
0                                          1.223   
1                                          1.223   
2                                          2.874   
3                                          2.874   
4                                          2.874   

   Margins of Supply Chain Emission Factors  \
0 

In [5]:
print(f"\n\nFactor Count: {factor_df.count()}")
print(f"\n\nActivity Count: {activity_df.count()}")



Factor Count: 2017 NAICS Code                                  1016
2017 NAICS Title                                 1016
GHG                                              1016
Unit                                             1016
Supply Chain Emission Factors without Margins    1016
Margins of Supply Chain Emission Factors         1016
Supply Chain Emission Factors with Margins       1016
Reference USEEIO Code                            1016
dtype: int64


Activity Count: business_activity_description    4998
Cost                             4998
Vendor                           4998
Comment                          4998
2017 NAICS Title                 4998
dtype: int64


In [ ]:
# import torch

In [ ]:
# device = torch.device('cpu')

In [6]:
label_dict = {value: idx for idx, value in enumerate(activity_df['2017 NAICS Title'].unique())}
activity_df['label'] = activity_df['2017 NAICS Title'].map(label_dict)


In [7]:
# print(activity_df)

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Split data into features and labels
X = activity_df['business_activity_description']  # Feature
y = activity_df['label']  # Assuming 'label' is already encoded as numeric labels

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the TF-IDF vectorizer on training data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Initialize and fit the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters as needed
clf.fit(X_train_tfidf, y_train)

# Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Make predictions on the test data
y_pred = clf.predict(X_test_tfidf)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# You can now use clf to make predictions on new data using the same tfidf_vectorizer to transform the new data



Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        22
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        23
           3       1.00      1.00      1.00        22
           4       1.00      1.00      1.00        26
           5       1.00      1.00      1.00        25
           6       1.00      1.00      1.00        27
           7       1.00      1.00      1.00        23
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00        20
          10       1.00      1.00      1.00        22
          11       1.00      1.00      1.00        18
          12       1.00      1.00      1.00        24
          13       1.00      1.00      1.00        26
          14       1.00      1.00      1.00        23
          15       1.00      1.00      1.00        23
          16       1.00      1.00      1.00        27
   

In [9]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer  # Assuming you've already fitted this with your training data

# Assuming clf is your trained RandomForestClassifier and tfidf_vectorizer is your fitted TF-IDF vectorizer

# Load the test data
activity_test_data = drive.CreateFile({'id': '1Q6Sm-lxT-cpOJpFeLg_Leyj3w28Ff9GY'})
activity_test_data.GetContentFile('business_activities_test_data.csv')
test_df = pd.read_csv('business_activities_test_data.csv')
test_df['encoded_labels'] = test_df['2017 NAICS Title'].apply(lambda x: label_dict.get(x, -1))  # Unseen labels get -1

# Prepare the test data features and labels
X_test_new = test_df['business_activity_description']  # Feature
y_test_new = test_df['encoded_labels']  # Replace 'label' with the actual column name for labels in your test data

# Transform the test data using the already fitted TF-IDF vectorizer
X_test_new_tfidf = tfidf_vectorizer.transform(X_test_new)

# Make predictions on the new test data
y_pred_new = clf.predict(X_test_new_tfidf)

# Evaluate the model on the new test data
print("New Test Data - Classification Report:\n", classification_report(y_test_new, y_pred_new))
print("New Test Data - Accuracy:", accuracy_score(y_test_new, y_pred_new))


New Test Data - Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        27
           2       1.00      1.00      1.00        24
           3       1.00      1.00      1.00        16
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        27
           6       1.00      1.00      1.00        16
           7       1.00      1.00      1.00        18
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00        23
          10       1.00      1.00      1.00        25
          11       1.00      1.00      1.00        27
          12       1.00      1.00      1.00        18
          13       1.00      1.00      1.00        22
          14       1.00      1.00      1.00        18
          15       1.00      1.00      1.00        21
          16       1.00      1.00      1.